Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [2]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [3]:
def x_y_split(df):
    target = 'status_group'
    X = df.copy().drop(target, axis=1)
    y = df.copy()[target]
    return X, y

X_train, y_train = x_y_split(train)
X_test = test

In [4]:
from sklearn.base import TransformerMixin
import category_encoders as ce

class Wrangler(TransformerMixin):

    def __init__(self):
        pass

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X):
        X = X.copy()
        X['latitude'] = X['latitude'].replace(-2e-08, 0)
        cols_with_zeros = ['longitude', 'latitude', 'gps_height', 'population', 'construction_year']
        for col in cols_with_zeros:
            X[col] = X[col].replace(0, np.nan)
        X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
        X['year'] = X['date_recorded'].dt.year
        X['month'] = X['date_recorded'].dt.month
        X['day'] = X['date_recorded'].dt.day
        X = X.drop(columns=['quantity_group', 'quality_group', 'payment_type', 'management_group', 'recorded_by', 'id', 'region', 'recorded_by', 'extraction_type', 'date_recorded'])
        return X



class NumericalFilter(TransformerMixin):

    def __init__(self, include = True):
        self.columns = None
        self.include = include

    def fit(self, X, y=None,**fit_params):
        if self.include:
            self.columns = X.select_dtypes(include='number').columns.tolist()
        else:
            self.columns = X.select_dtypes(exclude='number').columns.tolist()
        return self

    def transform(self, X):
        return X[self.columns]

In [5]:
from sklearn.pipeline import make_pipeline, make_union
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
import lightgbm
from lightgbm import LGBMClassifier
import numpy as np

categorical_pipe = make_pipeline(NumericalFilter(False), ce.OrdinalEncoder())

numerical_pipe = make_pipeline(NumericalFilter(), SimpleImputer(add_indicator=True))

processing_pipe = make_union(categorical_pipe, numerical_pipe)

classifier = make_pipeline(
    Wrangler(),
    processing_pipe,
    LGBMClassifier()
    )

In [6]:
processing_pipe

FeatureUnion(n_jobs=None,
             transformer_list=[('pipeline-1',
                                Pipeline(memory=None,
                                         steps=[('numericalfilter',
                                                 <__main__.NumericalFilter object at 0x7f8ab75fd750>),
                                                ('ordinalencoder',
                                                 OrdinalEncoder(cols=None,
                                                                drop_invariant=False,
                                                                handle_missing='value',
                                                                handle_unknown='value',
                                                                mapping=None,
                                                                return_df=True,
                                                                verbose=0))],
                                         verbose=False)),
     

In [7]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

In [8]:
paramaters = {
    'lgbmclassifier__num_leaves' : randint(10, 500 ),
    'lgbmclassifier__min_data_in_leaf': randint(100, 10000),
    'lgbmclassifier__max_depth': randint(20, 100),
    'featureunion__pipeline-2__simpleimputer__strategy': ['mean', 'median']
}

In [9]:
search = RandomizedSearchCV(
    classifier,
    param_distributions= paramaters,
    n_iter = 25,
    cv=10,
    return_train_score=True,
    n_jobs=-1,
    refit=True
)

In [10]:
search.fit(X_train, y_train)

RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('wrangler',
                                              <__main__.Wrangler object at 0x7f8ab37ecc90>),
                                             ('featureunion',
                                              FeatureUnion(n_jobs=None,
                                                           transformer_list=[('pipeline-1',
                                                                              Pipeline(memory=None,
                                                                                       steps=[('numericalfilter',
                                                                                               <__main__.NumericalFilter object at 0x7f8ab75fd750>),
                                                                                              ('ordinalencoder',
                                                     

In [11]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation MAE', search.best_score_)

Best hyperparameters {'featureunion__pipeline-2__simpleimputer__strategy': 'median', 'lgbmclassifier__max_depth': 85, 'lgbmclassifier__min_data_in_leaf': 409, 'lgbmclassifier__num_leaves': 445}
Cross-validation MAE 0.7962121212121213


In [27]:
df = pd.DataFrame(search.predict(X_test), index=X_test.index, columns=['status_group'])

In [28]:
X_test[['id']].merge(df, left_index=True, right_index=True).to_csv('submission.csv', index=False)